# Share map tiles with several datasets

In this notebook, you will be guided to :
- Create a **project** and upload an orthomosaic **composed of 2 rasters**
- Generate a URL template to download tiles displaying both rasters
- Visualize the tiles in the notebook
- Manage **share tokens** giving access to the raster

### You may execute this notebook one cell after the other with `Shift + Enter`

For more options, explore `Cell` in the top menu bar, or read a [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

## Requirements

- Alteia Python SDK
- folium
- shapely
- pyproj
- tabulate

⚠️ **Restart the Jupyter kernel after first install**

In [ ]:
!pip install alteia folium shapely pyproj tabulate

In [ ]:
import os
import alteia

In [ ]:
import getpass
platform_url = 'https://app.alteia.com'
login = input('Enter your email ')
password = getpass.getpass('Enter your password ')

In [ ]:
sdk = alteia.SDK(url=platform_url, user=login, password=password)

### Download sample files (images, mesh, raster...)

An archive `Split-orthomosaic.zip` containing sample files will be downloaded (if not found in the current directory).

In [ ]:
import urllib.request
import zipfile

try: working_dir
except NameError: working_dir = os.getcwd()
%cd {working_dir}

name = 'Split-orthomosaic'
filename = f'{name}.zip'

if not os.path.exists(name):
    print(f'"{name}" folder not found')
    if not os.path.exists(filename):
        print(f'{filename} not found')
        print('Downloading it...', end=' ')
        url = f'https://delair-transfer.s3-eu-west-1.amazonaws.com/sdks/sample-data/{filename}'
        filepath, _ = urllib.request.urlretrieve(url, filename)
        print('OK')

    print(f'Extracting "{filename}"...', end=' ')
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall('.')
    print('OK')
else:
    print(f'"{name}" folder found. No need to download it again.')

In [ ]:
sample_path = f'./{name}'

In [ ]:
%cd {sample_path}
!ls .

# Create the project

In [ ]:
import ipywidgets as widgets
my_companies = list(sdk.companies.search_generator())
print('Select the company to create your project :')
radio_buttons = widgets.RadioButtons(
    options=[c.name for c in my_companies],
)
radio_buttons

In [ ]:
company = my_companies[radio_buttons.index]

In [ ]:
my_project = sdk.projects.create(
    name='Share Map with several datasets',
    company=company.id,
    geometry={"coordinates": [[
                [2.0362935018049213, 43.33793077500704],
                [2.052168442639523, 43.33793077500704],
                [2.052168442639523, 43.34757088135606],
                [2.0362935018049213, 43.34757088135606],
                [2.0362935018049213, 43.33793077500704]]],
              "type": "Polygon"})

In [ ]:
print('We just created the project {!r} with id {!r}'.format(
    my_project.name, my_project.id))

## Create and upload the 2 rasters

In [ ]:
left_dataset = sdk.datasets.create_raster_dataset(
  name='Left',
  project=my_project.id,
  dataset_format='geotiff',
  categories=['orthomosaic'])

sdk.datasets.upload_file(
  dataset=left_dataset.id,
  component='raster',
  file_path='Left.tif')

print('"Left" raster uploaded')

right_dataset = sdk.datasets.create_raster_dataset(
  name='Right',
  project=my_project.id,
  dataset_format='geotiff',
  categories=['orthomosaic'])

sdk.datasets.upload_file(
  dataset=right_dataset.id,
  component='raster',
  file_path='Right.tif')

print('"Right" raster uploaded')

#### After the raster uploads, they need to be ingested. It can take up to 5 minutes.

In [ ]:
from time import sleep
print('Please wait till the rasters are ingested (it can takes up to 5 minutes)...')
while True:
    ds_list = sdk.datasets.describe([left_dataset.id, right_dataset.id])
    ingestion_statuses = [ds.ingestion.get('status') for ds in ds_list]
    if ingestion_statuses == ['completed', 'completed']:
        break
    else:
        sleep(10)

print('OK 👍')
print('The 2 rasters have been ingested properly')

# Tile layer URLs Generation

**Alteia** provides a tiling service for visualization of raster datasets.

It follows the TMS standard (for example: https://app.alteia.com/tileserver/tiles/DATASET_ID/{z}/{x}/{y}.png).

The access to this service resources is protected using a token.

The Python SDK provides the `sdk.datasets.share_tiles` function:
- to generate a token giving access to some datasets
- and build a URL template to download tiles displaying those datasets.

In [ ]:
map_tile_url = sdk.datasets.share_tiles(dataset=[left_dataset.id, right_dataset.id])
map_tile_url

# Visualization with Folium

[Folium](http://python-visualization.github.io/folium/) is a Python integration of an interactive map and provides easy visualization of geolocalized data.

To set the default location of the map, one must first determine the center of the data to visualize.

### Determine the center of the raster and its bounding box

In [ ]:
# Request the updated dataset properties (such as its geometry)
datasets = sdk.datasets.describe([left_dataset.id, right_dataset.id])

In [ ]:
from shapely.geometry import shape
from shapely.ops import cascaded_union

def merge_geometries(geometries):
    if not isinstance(geometries, list):
        geometries = [geometries]
    shapely_union_geometry = cascaded_union(geometries)
    return shapely_union_geometry

merged_geometry = merge_geometries([shape(ds.geometry) for ds in datasets])

In [ ]:
def get_center_coords(geometry):
    return list(geometry.centroid.coords)[0]

center_longitude, center_latitude = get_center_coords(merged_geometry)
map_center = (center_latitude, center_longitude)
print('Coordinates of the center: lat={:.4f}, long={:.4f}'.format(center_latitude, center_longitude))

In [ ]:
def get_bbox(geometry):
    return geometry.bounds  # (minx, miny, maxx, maxy)

bbox = get_bbox(merged_geometry)
print('Bounding box: {}'.format(bbox))

## Orthomosaic

In [ ]:
import folium

m = folium.Map(location=map_center, zoom_start=18)

folium.raster_layers.TileLayer(
    tiles=map_tile_url,
    attr='Alteia',
    max_zoom=20,
    overlay=False,
    control=True,
    bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]]
).add_to(m)
m

# Manage share tokens

When sharing map tiles with the functions above, a unique token is generated and included in the URL.
It is called a **share token**.
These tokens give access to specific datasets to the people or application you shared it with, without having to login on Alteia.

In order to control the share tokens that have been created, and to revoke some if necessary, please follow the following parts.

### List share tokens

In [ ]:
from datetime import datetime
from IPython.display import HTML, display
from tabulate import tabulate

headers = ['Index', 'Shared datasets', 'Company', 'Token creation', 'Token expiration', 'Token value']
table = []
for index, share_token in enumerate(sdk.share_tokens.search_generator()):
    shared_datasets = share_token.scope["datasets"].keys()
    table.append([
        index+1,
        '\n'.join(shared_datasets),
        share_token.company,
        share_token.creation_date,
        getattr(share_token, 'expiration_date', 'None'),
        share_token.token
    ])

display(HTML(tabulate(table, headers, tablefmt='html', colalign='left')))

### Revoke a share token

In [ ]:
import ipywidgets as widgets

share_tokens = list(sdk.share_tokens.search_generator())
print('Select an access token to revoke :')
radio_buttons = widgets.RadioButtons(
    options=[f'{index+1}) {token.token[:30]}...' for index, token in enumerate(share_tokens)],
)
radio_buttons

In [ ]:
token_to_revoke = share_tokens[radio_buttons.index]

In [ ]:
print(f'Revoking token {token_to_revoke.token}...')
sdk.share_tokens.revoke(token_to_revoke.token)
print()
print('OK 👍')